In [104]:
import sys
import math
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install texttable
!{sys.executable} -m pip install matplotlib

In [127]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from texttable import Texttable

df = pd.read_csv('/Users/sanjaychari/Documents/RPI/Tachyon_Research/ganglia_database/rrds/Manager/novadaq-far-datadisk-04.fnal.gov/novadaq-far-datadisk-04.fnal.gov.csv')
df.fillna(0, inplace=True)

zero_cols = df.columns[(df == 0).all()]
df.drop(labels=zero_cols, axis=1, inplace=True)

In [128]:
X = df.drop(columns=['Epoch Timestamp', 'FTS failed transfers', 'FTS merged-log failed transfers'])
y = df['FTS failed transfers']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [129]:
def linear_regression(X_train, y_train, X_test, y_test):
    # Create linear regression object
    regr = linear_model.LinearRegression()
    
    # Train the model using the training sets
    regr.fit(X_train, y_train)
    
    # Make predictions using the testing set
    y_pred = regr.predict(X_test)
    
    # The coefficients
    print("Coefficients: \n", regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f" % mean_squared_error(y_test.to_list(), y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print("Coefficient of determination: %.2f" % r2_score(y_test.to_list(), y_pred))

    return regr

regr = linear_regression(X_train, y_train, X_test, y_test)

Coefficients: 
 [ 1.46921255e-02  8.22851298e-01 -1.13952809e-01  3.54351587e-03
 -2.30584431e-03  7.85558378e-13  4.48016759e-07 -1.24328311e+01
  2.30292347e-03  2.85903700e-01 -1.30415919e-02 -1.09414042e-02
  1.30638928e-01 -1.08225462e-01  4.73202001e-04 -6.80332803e-06
 -5.09706753e-08  2.77074233e-03  5.17399969e-04 -2.06420367e-05
 -7.38103633e-13 -2.76100313e-05  6.92538341e-01 -4.12957414e-03
  2.16119557e-05 -1.44422160e-06  1.19981800e-02 -8.16705965e-03
  6.41239999e-13  8.02227135e-05 -2.75487382e-02  1.68435784e-03
 -9.91077696e-08 -3.05471715e-08 -3.16057972e-03  1.80398085e-08
  5.33352664e-07 -5.18325088e-13  3.40854278e-02 -3.63848527e-06
  6.52691308e-01  2.94832639e-07  3.41086770e-01 -1.88509983e-01
 -1.64064095e-03  6.87667748e-05 -4.83769355e-04 -1.43750404e-02
 -1.97714493e-05]
Mean squared error: 3.27
Coefficient of determination: 1.00


In [130]:
orig_coef_dic = {}
abs_coef_dic = {}

for i in range(len(X.columns)):
    orig_coef_dic[X.columns[i]] = regr.coef_[i]
    abs_coef_dic[X.columns[i]] = abs(regr.coef_[i])

most_significant_coefs = sorted(abs_coef_dic.items(), key=lambda item: -item[1])[:10]
signed_most_sig_coefs = []

for ele in most_significant_coefs:
    signed_most_sig_coefs.append([ele[0], orig_coef_dic[ele[0]]])

In [131]:
print("Most Influential Fields on FTS failed transfers considering other failures")
print("-"*75)
table = Texttable()
signed_most_sig_coefs.insert(0, ['Field', 'Linear Regression Coefficient'])
table.add_rows(signed_most_sig_coefs)
print(table.draw())

Most Influential Fields on FTS failed transfers considering other failures
---------------------------------------------------------------------------
+------------------------------+-------------------------------+
|            Field             | Linear Regression Coefficient |
+==============================+===============================+
| cpu_aidle                    | -12.433                       |
+------------------------------+-------------------------------+
| FTS merged-log failed files  | 0.823                         |
+------------------------------+-------------------------------+
| FTS merged-log pending files | 0.693                         |
+------------------------------+-------------------------------+
| FTS merged-log new files     | 0.653                         |
+------------------------------+-------------------------------+
| swap_total                   | 0.341                         |
+------------------------------+-------------------------------+
| pa

In [132]:
drop_columns = ['Epoch Timestamp']
drop_columns.extend([column for column in df.columns if "failed" in column])
X = df.drop(columns=drop_columns)
y = df['FTS failed transfers']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [133]:
regr = linear_regression(X_train, y_train, X_test, y_test)

Coefficients: 
 [ 1.25727815e-01  1.52607495e-02  1.24658484e-02  2.77609395e-12
 -1.43882683e+01  2.68699046e+00 -9.53220560e-03 -1.49691575e-03
  6.94776021e-01 -4.53259456e+00 -2.96192070e-03 -3.70918406e-07
  1.52971635e+00  7.50239044e-03 -7.79114456e-04 -2.60702833e-12
 -7.75557578e-05 -4.21982280e-02  1.10217479e-02  5.76790125e-04
  4.20696818e-06  1.74837908e-01 -2.86798825e-01  2.26821355e-12
  1.66794796e-04 -2.20466289e+00  1.63059708e-03 -2.24218167e-07
 -6.05613311e-07  3.86173069e-02  2.65639014e-08  1.87968366e-06
 -1.84649464e-12 -6.72525163e-01 -1.66636523e-05  5.36405672e-01
  2.40897125e-06 -1.56752696e+00 -1.45323111e-03 -3.42101805e-02
  3.15611055e-03 -1.60111570e-01 -3.38133575e-04]
Mean squared error: 26.61
Coefficient of determination: 0.96


In [134]:
orig_coef_dic = {}
abs_coef_dic = {}

for i in range(len(X.columns)):
    orig_coef_dic[X.columns[i]] = regr.coef_[i]
    abs_coef_dic[X.columns[i]] = abs(regr.coef_[i])

most_significant_coefs = sorted(abs_coef_dic.items(), key=lambda item: -item[1])[:10]
signed_most_sig_coefs = []

for ele in most_significant_coefs:
    signed_most_sig_coefs.append([ele[0], orig_coef_dic[ele[0]]])

In [135]:
print("Most Influential Fields on FTS failed transfers without considering other failures")
print("-"*85)
table = Texttable()
signed_most_sig_coefs.insert(0, ['Field', 'Linear Regression Coefficient'])
table.add_rows(signed_most_sig_coefs)
print(table.draw())

Most Influential Fields on FTS failed transfers without considering other failures
-------------------------------------------------------------------------------------
+--------------------------+-------------------------------+
|          Field           | Linear Regression Coefficient |
+==========================+===============================+
| cpu_aidle                | -14.388                       |
+--------------------------+-------------------------------+
| FTS rawA new files       | -4.533                        |
+--------------------------+-------------------------------+
| part_max_used            | 2.687                         |
+--------------------------+-------------------------------+
| load_fifteen             | -2.205                        |
+--------------------------+-------------------------------+
| swap_total               | -1.568                        |
+--------------------------+-------------------------------+
| load_five                | 1.530    

In [136]:
X = df[['bytes_in']]
y = df['cpu_aidle']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [137]:
regr = linear_regression(X_train, y_train, X_test, y_test)

Coefficients: 
 [5.77042496e-06]
Mean squared error: 226.27
Coefficient of determination: 0.88
